# Bias Variance

![title](ml_bias_variance.png)

Мы не будем выписывать строгие формулы, но попробуем объяснить идею этих понятий.

Пусть у нас есть алгоритм обучения, который по данным может создать модель.

Ошибка этих моделей может быть разложена на три части:
* **Noise** – шум данных, не предсказуем, теоретический минимум ошибки
* **Bias** – смещение, на сколько хорошо работает средний алгоритм. Средний алгоритм это "возьмём случайные данные, обучим алгоритм, сделаем предсказания", **Bias** – это ошибка средних предсказаний.
* **Variance** – разброс, на сколько устойчиво работает алгоритм. Опять же "возьмём случайные данные, обучим алгоритм, сделаем предсказания", **Variance** – это разрос этих предсказаний.

# Бустинг и Бэггинг в терминах Bias и Variance

Как вы думаете на какую составляющую Бустинг и Бэггинг влияют, а на какую нет?

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('HR.csv')

In [3]:
data.head()

,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years
0,0.53,2,157,3,0,1,0
1,0.86,5,262,6,0,0,0
2,0.88,7,272,4,0,1,0
3,0.87,5,223,5,0,1,0
4,0.52,2,159,3,0,1,0


In [4]:
X, y = data.drop('left', axis=1).values, data['left'].values

In [5]:
data['left'].unique()

array([1, 0])

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [7]:
def sample_model(seed, model):
    random_gen = np.random.RandomState(seed)
    indices = random_gen.choice(len(y_train), size=len(y_train), replace=True)
    model.fit(X_train[indices, :], y_train[indices])
    return model

def estimate_bias_variance(model, iters_count=100):
    y_preds = []
    for seed in range(iters_count):
        model = sample_model(seed, model)
        y_preds.append(model.predict(X_test))
    y_preds = np.array(y_preds)
    
    print('Bias:', np.mean((y_test - y_preds.mean(axis=0)) ** 2))
    print('Variance:', y_preds.std(axis=0).mean())

**Линейная регрессия**

In [8]:
estimate_bias_variance(LinearRegression())

Bias: 0.22539321164615467
Variance: 0.010711666687293465


**Решающее дерево с max_depth=5**

In [9]:
estimate_bias_variance(DecisionTreeRegressor(max_depth=5))

Bias: 0.17343635344369013
Variance: 0.04434523236701086


**Решающее дерево с max_depth=10**

In [10]:
estimate_bias_variance(DecisionTreeRegressor(max_depth=10))

Bias: 0.17175575739495175
Variance: 0.11712092704487344


**Решающее дерево с max_depth=15**

In [11]:
estimate_bias_variance(DecisionTreeRegressor(max_depth=15))

Bias: 0.17842598190450087
Variance: 0.21661949936646008


**Решающее дерево без ограничения глубины**

In [12]:
estimate_bias_variance(DecisionTreeRegressor(max_depth=None))

Bias: 0.2069107045423811
Variance: 0.32457384418180296


**Случайный лес n_estimators=1**

In [13]:
estimate_bias_variance(RandomForestRegressor(n_estimators=1, random_state=42))

Bias: 0.19463122486057333
Variance: 0.35705073628637773


**Случайный лес n_estimators=10**

In [14]:
estimate_bias_variance(RandomForestRegressor(n_estimators=10, random_state=42))

Bias: 0.19311294566535084
Variance: 0.17229587181057013


**Случайный лес n_estimators=50**

In [15]:
estimate_bias_variance(RandomForestRegressor(n_estimators=50, random_state=42))

Bias: 0.19315888675365975
Variance: 0.14255099142514835


**XGBRegressor**

**Бустинг над деревьями max_depth=20**

In [16]:
estimate_bias_variance(XGBRegressor(n_estimators=1, max_depth=20))

Bias: 0.23515768239943852
Variance: 0.022880817


**Бустинг над деревьями max_depth=10**

In [17]:
estimate_bias_variance(XGBRegressor(n_estimators=1, max_depth=10))

Bias: 0.23460312600664116
Variance: 0.01066339


**Бустинг над деревьями max_depth=5**

In [18]:
estimate_bias_variance(XGBRegressor(n_estimators=1, max_depth=5))

Bias: 0.23539367931741623
Variance: 0.004351252


**Бустинг над деревьями n_estimators=10**

In [19]:
estimate_bias_variance(XGBRegressor(n_estimators=10, max_depth=5))

Bias: 0.18128282857388248
Variance: 0.019852929


**Бустинг над деревьями n_estimators=100**

In [20]:
estimate_bias_variance(XGBRegressor(n_estimators=100, max_depth=5))

Bias: 0.17182469278418883
Variance: 0.05643562


**CatBoostRegressor**

In [21]:
estimate_bias_variance(CatBoostRegressor(n_estimators=1, max_depth=6, verbose=False))

Bias: 0.3467385908579134
Variance: 0.0006835754697775419


In [22]:
estimate_bias_variance(CatBoostRegressor(n_estimators=10, max_depth=6, verbose=False))

Bias: 0.2801365481999864
Variance: 0.0058176648330751516


In [23]:
estimate_bias_variance(CatBoostRegressor(n_estimators=100, max_depth=6, verbose=False))

Bias: 0.17608858395709903
Variance: 0.019651480967338052


**LGBMRegressor**

In [24]:
estimate_bias_variance(LGBMRegressor(n_estimators=1, max_depth=5))

Bias: 0.2193821065837484
Variance: 0.006619492577068203


In [25]:
estimate_bias_variance(LGBMRegressor(n_estimators=10, max_depth=5))

Bias: 0.18007513755165383
Variance: 0.020490996273996646


In [26]:
estimate_bias_variance(LGBMRegressor(n_estimators=100, max_depth=5))

Bias: 0.1717763977728146
Variance: 0.054249435185578586
